In [ ]:
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', None)
from tqdm import tqdm
import random
from pprint import pprint
import re
import copy
import json

In [ ]:
from openai import OpenAI
client = OpenAI()
import time
SLEEP_TIME = 1

def gpt4(input_text, prior_messages=None):
    gpt4_kwargs = {
        "model": "gpt-4-turbo",
        "temperature": 0,
    }
    if prior_messages is None:
        messages = [{"role": "system", "content": "You are a helpful assistant."}]
    else:
        messages = copy.deepcopy(prior_messages)
    messages.append({"role": "user", "content": input_text})
    while True:
        try:
            response = client.chat.completions.create(messages=messages, **gpt4_kwargs)
            break
        except Exception as e:
            print("-"*30)
            print(e)
            if type(e).__name__ == "RateLimitError":
                print(f"Sleep for {SLEEP_TIME}......")
                time.sleep(SLEEP_TIME)
            print("-"*30)
    output_text = response.choices[0].message.content
    messages.append({"role": "assistant", "content": output_text})
    return output_text, messages

In [ ]:
measured_person = ["PERSON-1", "PERSON-2", "PERSON-3", "PERSON-4"]

In [ ]:
def load_data(path):
    excel_file = pd.ExcelFile(path)
    sheet_names = excel_file.sheet_names
    all_sheets_data = {sheet: excel_file.parse(sheet) for sheet in sheet_names}
    return all_sheets_data

In [ ]:
raw_data = load_data("annotation_merged_v2.xlsx")

In [ ]:
prompt_template = """Based on the sub-question’s relevance and functional role in answering the complex question, classify the sub-question into three types: core, background, and follow-up.

The definitions of these three sub-question types are:
(1) Core sub-questions:
 - They are central to the main topic and directly or partially address the complex question.
 - They are crucial for interpreting the logical reasoning of the complex question and provide essential insights required for answering the complex question.
 - They often involve multiple steps or perspectives, making them fundamental to generating a comprehensive and well-rounded response to the complex question.
(2) Background sub-questions:
 - They are optional when answering the complex question, but they can provide additional context or background information that helps clarify the complex question.
 - Their primary role is to support the understanding of the main topic by offering supplementary evidence or information, though it is not strictly necessary for addressing the core aspects of the complex question.
(3) Follow-up sub-questions:
 - They are not needed to answer the complex question.
 - They often arise after users receive an initial answer and seek further clarification or details.
 - They may explore specific aspects of the response in greater depth, but their answers can sometimes be out-of-scope or beyond the focus of the original complex question.

Organize the answer in JSON-format, shaped as {"type": "core"}, {"type": "background"}, or {"type": "follow-up"}.
Place the JSON-format answer between <answer> and </answer> tags.

Here are a few examples you can use for reference:

$FEW-SHOT-EXAMPLES

Complex question: $QUESTION
Its sub-question: $SUB-QUESTION
Answer in JSON-format:
"""

In [ ]:
def majority_vote(row):
    values, counts = np.unique(row, return_counts=True)
    max_count_index = np.argmax(counts)
    return values[max_count_index]

for i in range(1, 11):
    df = raw_data[f"q{i}"]
    df['Question'] = df['Question'][0]
    df['majority_vote'] = df[['PERSON-1', 'PERSON-2', 'PERSON-3', 'PERSON-4']].apply(majority_vote, axis=1)

HARD = ["q1", "q2", "q3", "q4", "q5", "q6", "q7", "q8", "q9", "q10"]
def prepare_few_shot_examples():
    all_examples = list()
    for tab in HARD:
        df = raw_data[tab]
        for _, row in df.iterrows():
            question = row["Question"]
            sub_question = row["Sub-Question"]
            label = row["majority_vote"]
            example = "Complex question: $QUESTION\nIts sub-question: $SUB-QUESTION\nAnswer in JSON-format: \{\"type\": \"$LABEL\"\}"
            example = example.replace("$QUESTION", question).replace("$SUB-QUESTION", sub_question).replace("$LABEL", label)
            all_examples.append(example)
    return "\n\n".join(all_examples)

few_shot_examples = prepare_few_shot_examples()

def prompt(question, sub_question):
    raw_pred, _ = gpt4(prompt_template.replace("$QUESTION", question).replace("$SUB-QUESTION", sub_question).replace("$FEW-SHOT-EXAMPLES", few_shot_examples))
    try:
        parsed_pred = json.loads(raw_pred.replace("<answer>", "").replace("</answer>", "").strip())
        prediction = parsed_pred["type"]
    except:
        print(raw_pred)
        prediction = "None"
    return prediction

In [ ]:
with open("decompositions.json", "r") as f:
    question_dataset = json.load(f)

question_dataset_with_type = list()
for sample in tqdm(question_dataset):
    question = sample["question"]
    sample_with_type = {"question": question, "sub_questions_with_types": []}
    for sub_question in sample["sub_questions"]:
        sub_q_type = prompt(question, sub_question)
        sample_with_type["sub_questions_with_types"].append({"sub_question": sub_question, "type": sub_q_type})
    question_dataset_with_type.append(sample_with_type)

with open("decompositions_with_type.json", "w") as f:
    json.dump(question_dataset_with_type, f, indent=4)